In [1]:
f = open("english_python_data.txt", "r",encoding="utf8")
file_lines = f.readlines()

In [2]:
dps = []
dp = None
for line in file_lines:
  if line[0] == "#":
    if dp:
      dp['solution'] = ''.join(dp['solution'])
      dps.append(dp)
    dp = {"question": None, "solution": []}
    dp['question'] = line[1:]
  else:
    dp["solution"].append(line)

In [3]:
print("Dataset size:", len(dps))

Dataset size: 4521


In [4]:
import keyword

print(keyword.kwlist)

['False', 'None', 'True', '__peg_parser__', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']


In [5]:
import random
import io
from tokenize import tokenize, untokenize
def augment_tokenize_python_code(python_code_str, mask_factor=0.3):


    var_dict = {} # Dictionary that stores masked variables

    # certain reserved words that should not be treated as normal variables and
    # hence need to be skipped from our variable mask augmentations
    skip_list = ['range', 'enumerate', 'print', 'ord', 'int', 'float', 'zip'
                 'char', 'list', 'dict', 'tuple', 'set', 'len', 'sum', 'min', 'max']
    skip_list.extend(keyword.kwlist)

    var_counter = 1
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []

    for i in range(0, len(python_tokens)):
      if python_tokens[i].type == 1 and python_tokens[i].string not in skip_list:

        if i>0 and python_tokens[i-1].string in ['def', '.', 'import', 'raise', 'except', 'class']: # avoid masking modules, functions and error literals
          skip_list.append(python_tokens[i].string)
          tokenized_output.append((python_tokens[i].type, python_tokens[i].string,python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
        elif python_tokens[i].string in var_dict:  # if variable is already masked
          tokenized_output.append((python_tokens[i].type, var_dict[python_tokens[i].string],python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
        elif random.uniform(0, 1) > 1-mask_factor: # randomly mask variables
          var_dict[python_tokens[i].string] = 'var_' + str(var_counter)
          var_counter+=1
          tokenized_output.append((python_tokens[i].type, var_dict[python_tokens[i].string],python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
        else:
          skip_list.append(python_tokens[i].string)
          tokenized_output.append((python_tokens[i].type, python_tokens[i].string,python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))

      else:
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string,python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
    return untokenize(tokenized_output).decode('utf-8')

In [6]:
dps2 = []
train_expansion_factor = 4
for j in range(train_expansion_factor):
  for i in range(len(dps)):
      try:
          dp={}
          dp['question']=dps[i]['question']
          dp['solution']=augment_tokenize_python_code(dps[i]['solution'])
          dps2.append(dp)
      except:
          pass


In [7]:
print(len(dps2))

18084


In [8]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers.trainers import WordLevelTrainer

# Define a custom pre-tokenizer to handle Python code
class PythonPreTokenizer():
    def token(self,i, text):
        tokenized_output2=[]
        row=row_sum=prev_row=0
        python_tokens = list(tokenize(io.BytesIO(str(text).encode('utf-8')).readline))
        # print(python_tokens)
        for i in range(0, len(python_tokens)):        
            type_token,token,(srow,scol),(erow,ecol),line=python_tokens[i]
            # print(token,srow,scol,erow,ecol)
            if srow>row:
                row+=1
                row_sum+=prev_row
            if token:
                tokenized_output2.append(text[row_sum+scol:row_sum+ecol])
            prev_row=len(line)
        return tokenized_output2 if len(tokenized_output2) else [text]
    
    def pre_tokenize(self, pretok):        
        pretok.split(self.token)

# Initialize a tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

# Set the pre-tokenizer to PythonPreTokenizer
tokenizer.pre_tokenizer = PreTokenizer.custom(PythonPreTokenizer())

# Tokenize a Python code snippet
string="""def add_two_numbers (num1 ,num2):
    sum =num1 +num2 
    return sum"""


In [9]:
def get_all_sentences(ds):
    for item in ds:
        if item['solution']:
            yield item['solution']

In [10]:
# def tokenize_python_code(python_code_str):
#     python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
#     tokenized_output = []
#     for i in range(0, len(python_tokens)):
#         if python_tokens[i].string=='x':
#             tokenized_output.append((python_tokens[i].type, 'var_1',python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
#             continue                        
#         tokenized_output.append((python_tokens[i].type, python_tokens[i].string,python_tokens[i].start,python_tokens[i].end,python_tokens[i].line))
#     return tokenized_output

In [11]:
# string2=dps[1736]['solution']
# python_tokens = list(tokenize(io.BytesIO(string2.encode('utf-8')).readline))
# # python_tokens[3]='var1'
# python_tokens2=tokenize_python_code(string2)
# print(untokenize(python_tokens2).decode('utf-8'))

In [12]:
# print(augment_tokenize_python_code(string2))

In [13]:
# print(dps2[1736]['solution'])

In [14]:
# augment_tokenize_python_code(dps2[1736]['solution'])

In [15]:
for x in get_all_sentences(dps[3662:3663]):
    print(tokenizer.pre_tokenizer.pre_tokenize_str(x))

[('\n', (1, 2)), ('def', (2, 5)), ('isBuzz', (6, 12)), ('(', (12, 13)), ('num', (13, 16)), (')', (16, 17)), (':', (18, 19)), ('\n', (20, 21)), ('\n', (23, 24)), ('    ', (24, 28)), ('return', (28, 34)), ('(', (35, 36)), ('num', (36, 39)), ('%', (40, 41)), ('10', (42, 44)), ('==', (45, 47)), ('7', (48, 49)), ('or', (50, 52)), ('num', (53, 56)), ('%', (57, 58)), ('7', (59, 60)), ('==', (61, 63)), ('0', (64, 65)), (')', (65, 66)), ('\n', (68, 69)), ('\n', (71, 72))]


In [16]:
s="""c =var_1 <<2 
print ("Binary Left Shift",c )"""
print(tokenizer.pre_tokenizer.pre_tokenize_str(s))

[('c', (0, 1)), ('=', (2, 3)), ('var_1', (3, 8)), ('<<', (9, 11)), ('2', (11, 12)), ('\n', (13, 14)), ('print', (14, 19)), ('(', (20, 21)), ('"Binary Left Shift"', (21, 40)), (',', (40, 41)), ('c', (41, 42)), (')', (43, 44))]


In [17]:
trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"])
tokenizer.train_from_iterator(get_all_sentences(dps2), trainer=trainer)

In [18]:
tokenizer.get_vocab_size()


6760

In [22]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [23]:
tokenizer.save('tokenizer_solution.json')

In [24]:
tok=Tokenizer.from_file('tokenizer_solution.json')

tok.pre_tokenizer = PreTokenizer.custom(PythonPreTokenizer())

In [25]:
tok.encode(string).ids

[18, 1384, 5, 92, 7, 104, 6, 8, 4, 16, 68, 9, 92, 26, 104, 4, 21, 68]

In [25]:
print(tok.decode([18, 1390, 5, 102, 7, 111, 6, 8, 4, 16, 68, 9, 102, 26, 111, 4, 21, 68]))

def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum


In [19]:
tokenizer.encode(string).ids

[18, 1384, 5, 92, 7, 104, 6, 8, 4, 16, 68, 9, 92, 26, 104, 4, 21, 68]

In [20]:
print(tokenizer.decode([18, 1384, 5, 92, 7, 104, 6, 8, 4, 16, 68, 9, 92, 26, 104, 4, 21, 68]))

def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum


In [ ]:
from tokenize import tokenize, untokenize
import io

def tokenize_python_code(python_code_str):
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output=[]
    tokenized_output2=[]
    row=0
    row_sum=0
    prev_row=0
    for i in range(0, len(python_tokens)):
        tokenized_output.append(python_tokens[i].string)
        type,token,(srow,scol),(erow,ecol),line=python_tokens[i]
        # print(type,token,srow,scol,erow,ecol)
        if srow>row:
            row+=1
            row_sum+=prev_row
        tokenized_output2.append(python_code_str[row_sum+scol:row_sum+ecol])
        prev_row=len(line)
    print(tokenized_output)
    print(tokenized_output2)
    return tokenized_output

# def handle_token(type,token,(srow,scol),(erow,ecol),line):
#     print(srow,scol,erow,ecol,tokenize.tok_name[type],repr(token))

In [ ]:
print(string[28])

In [ ]:
string="""def add_two_numbers (num1 ,num2):
    sum =num1 +num2 
    return sum"""


toks=tokenize_python_code(string)


In [1]:
from train import dataset_load

c:\Users\prabh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data=dataset_load()

In [3]:
from train import get_or_build_tokenizer
from config import get_config
config=get_config()

In [4]:

tokenizer_src = get_or_build_tokenizer(config, data, 'question')
tokenizer_tgt = get_or_build_tokenizer(config, data, 'solution')

In [5]:
tokenizer_tgt.encode(data[0]['solution']).ids

[97, 9, 636, 4, 107, 9, 688, 4, 68, 9, 97, 26, 107, 4, 13, 5, 1197, 6, 4, 4, 4]

In [6]:
print(tokenizer_tgt.decode([10, 9, 639, 4, 111, 9, 686, 4, 68, 9, 10, 26, 111, 4, 13, 5, 1215, 6, 4, 4, 4]))

var_1 = IplData 
 join = 'z' 
 sum = var_1 + join 
 print ( longest_non_repeat ) 
 
 



In [7]:
from train import BilingualDataset

In [8]:
train_ds = BilingualDataset(data, tokenizer_src, tokenizer_tgt, 'question', 'solution', config['seq_len'])


In [9]:
max_len_src = 0
max_len_tgt = 0
lst=[]

for item in data:
    src_ids = tokenizer_src.encode(item['question']).ids
    tgt_ids = tokenizer_tgt.encode(item['solution']).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))
    lst.append(len(tgt_ids))

print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')

Max length of source sentence: 81
Max length of target sentence: 387


In [1]:
import torch

In [2]:
torch.cuda.empty_cache() 

In [12]:
torch.cuda.get_device_properties(0).total_memory

4294639616

In [7]:
a = torch.ones(1, device="cuda") 

In [14]:
r = torch.cuda.memory_reserved(0)
av = torch.cuda.memory_allocated(0)

In [16]:
r

2097152